# Data Integration with BDI

In [1]:
from bdi import APIManager

/Users/rlopez/opt/anaconda3/envs/askem/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ['BDI_MODEL_PATH'] = 'YOUR PATH HERE'

In [3]:
manager = APIManager()

## Dataset Loading

In [4]:
dataset_path =  './datasets/dou.csv'
dataset = manager.load_dataset(dataset_path)
dataset

,idx,Proteomics_Participant_ID,Case_excluded,Proteomics_TMT_batch,Proteomics_TMT_plex,Proteomics_TMT_channel,Proteomics_Parent_Sample_IDs,Proteomics_Aliquot_ID,Proteomics_Tumor_Normal,Proteomics_OCT,...,RNAseq_R1_sample_type,RNAseq_R1_filename,RNAseq_R1_UUID,RNAseq_R2_sample_type,RNAseq_R2_filename,RNAseq_R2_UUID,miRNAseq_sample_type,miRNAseq_UUID,Methylation_available,Methylation_quality
0,S001,C3L-00006,No,2,5,128N,C3L-00006-01,CPT0001460012,Tumor,No,...,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_TAGCTT_S17...,8a1efc47-1c29-417f-a425-cdbd09565dcb,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_TAGCTT_S17...,8c3fe9b7-7acd-4867-8d9c-a8e5d1516eda,Tumor,37bcba98-1094-459e-83ae-c23a602416fb,YES,PASS
1,S002,C3L-00008,No,4,16,130N,C3L-00008-01,CPT0001300009,Tumor,No,...,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GGCTAC_S22...,555725e8-cba5-4676-9b0a-80100cbf9f47,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GGCTAC_S22...,15235b12-b67a-4678-acc4-ed03d642bd5e,Tumor,492b50d8-ec35-46e7-a65d-06512aaee394,YES,PASS
2,S003,C3L-00032,No,1,2,131,C3L-00032-01,CPT0001420009,Tumor,No,...,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GTCCGC_S18...,9ae968f3-691d-4db3-9977-1ab3e5af9085,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GTCCGC_S18...,423b6b09-02aa-4f47-9241-f75c1dad1161,Tumor,1794ff56-db2d-4d1a-8758-cab7fe3d98c1,YES,PASS
3,S004,C3L-00084,Yes,3,11,129N,C3L-00084-01,CPT0000820012,Tumor,No,...,Tumor,170818_UNC32-K00270_0050_AHL2FHBBXX_ATCACG_S5_...,b0a7cdf2-2ad8-4442-91b0-548ea4975554,Tumor,170818_UNC32-K00270_0050_AHL2FHBBXX_ATCACG_S5_...,c83987a5-1c13-4af4-b46c-218fe5f60c34,NaN,NaN,YES,PASS
4,S005,C3L-00090,No,3,12,129C,C3L-00090-01,CPT0001140003,Tumor,No,...,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GAGTGG_S10...,8ce5618d-9ff6-40f9-aeea-8d8e1633ae38,Tumor,170802_UNC31-K00269_0072_AHK3GVBBXX_GAGTGG_S10...,06d3fd4a-a623-4146-8500-4f1f17235253,Tumor,a6524c2d-d7dd-4629-980e-b45dbdc92c49,YES,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,S149,NX14,No,5,17,128C,"C3N-03699-01,C3N-03699-02","CPT0230130002,CPT0230130003,CPT0230130004,CPT0...",Enriched_normal,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,S150,NX15,No,5,17,131,C3N-03700-01,"CPT0230190002,CPT0230190003,CPT0230190004",Enriched_normal,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,S151,NX16,No,5,17,129C,"C3N-03701-01,C3N-03701-02,C3N-03701-03","CPT0230250002,CPT0230260002,CPT0230270002,CPT0...",Enriched_normal,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,S152,NX17,No,5,17,127C,"C3N-03706-01,C3N-03706-02,C3N-03706-03","CPT0230400002,CPT0230400003,CPT0230400004,CPT0...",Enriched_normal,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Column Mapping

In [5]:
manager.reduce_scope()
column_mappings = manager.map_columns()
column_mappings

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 179/179 [00:04<00:00, 42.76it/s]


Table features extracted from 179 columns


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 734/734 [00:17<00:00, 41.39it/s]


Table features extracted from 734 columns


{'idx': 'id',
 'Proteomics_Participant_ID': 'histologic_progression_type',
 'Case_excluded': 'procedures_performed',
 'Proteomics_TMT_batch': 'percent_tumor_cells',
 'Proteomics_TMT_plex': 'percent_tumor_cells',
 'Proteomics_TMT_channel': 'tissue_microarray_coordinates',
 'Proteomics_Parent_Sample_IDs': 'protein_expressions',
 'Proteomics_Aliquot_ID': 'prostatic_chips_positive_count',
 'Proteomics_Tumor_Normal': 'residual_tumor_measurement',
 'Proteomics_OCT': 'spectrophotometer_method',
 'Country': 'country_of_birth',
 'Histologic_Grade_FIGO': 'histologic_progression_type',
 'Myometrial_invasion_Specify': 'perineural_invasion_present',
 'Histologic_type': 'histologic_progression_type',
 'Treatment_naive': 'treatment_type',
 'Tumor_purity': 'tumor_purity',
 'Path_Stage_Primary_Tumor-pT': 'history_of_tumor',
 'Path_Stage_Reg_Lymph_Nodes-pN': 'lymph_nodes_tested',
 'Clin_Stage_Dist_Mets-cM': 'msi_status',
 'Path_Stage_Dist_Mets-pM': 'somatic_mutation_calling_workflows',
 'tumor_Stage-Pat

## Value Mapping

In [6]:
manager.map_values()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Column tumor_Stage-Pathological:
| Current Value   | Target Value   | Similarity         |
|-----------------+----------------+--------------------|
| Stage I         | Three Tier     | 1.0                |
| Stage IV        | Three Tier     | 0.9224806201550388 |
| Stage II        | Three Tier     | 0.9224806201550388 |
| Stage III       | Three Tier     | 0.8531211750305997 |
| nan             | -              | -                  | 

Column Race:
| Current Value             | Target Value              | Similarity   |
|---------------------------+---------------------------+--------------|
| White                     | white                     | 1.0          |
| Asian                     | asian                     | 1.0          |
| Not Reported              | not reported              | 1.0          |
| Black or African American | black or african american | 1.0          |
| nan                       | -                         | -            | 

Column Tumor_Focality:
| Current 